In [1]:
using QuantumCumulants
using IterTools
using Symbolics
using SymbolicUtils
using SymbolicUtils.Rewriters

include("../src/diagrams.jl")
include("../src/contractions.jl")
include("../src/lindblad.jl")
include("../src/printing.jl")
include("../src/poles.jl")

reshape_sols

In [2]:
N = 2 # number of atoms 
h = ⊗([NLevelSpace(Symbol(:atom,i),2) for i=1:N]...)
σ(i,j,k) = Transition(h,Symbol("σ_{$k}"),i,j,k)

σz1 = 2*σ(1,1,1) - 1
σz2 = 2*σ(1,1,2) - 1

σx1 = σ(1,2,1) + σ(2,1,1)
σx2 = σ(1,2,2) + σ(2,1,2)

#@show typeof(σz1)
@cnumbers ω_1 ω_2 ω_d g ϵ_d

#Define ordered lists of frequencies and hamiltonian terms 
ωs = [0, 0, 0, ω_d, -ω_d]
hs = [ω_1*σz1/2,ω_2*σz2/2, g* σx1 * σx2, ϵ_d* σx2,ϵ_d* σx2]

5-element Vector{QuantumCumulants.QAdd}:
 (ω_1+-ω_1*(σ_{1}22)+-0.5ω_1)
 (ω_2+-ω_2*(σ_{2}22)+-0.5ω_2)
 (g*(σ_{1}12*σ_{2}12)+g*(σ_{1}12*σ_{2}21)+g*(σ_{1}21*σ_{2}12)+g*(σ_{1}21*σ_{2}21))
 (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21))
 (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21))

In [25]:
k=2
@cnumbers t
H = simplify(effective_hamiltonian(k, ωs, hs,t))
secondOrderRule = @acrule((~a)*(~y)*(~z) + (~b)*(~y)*(~z) => ((~a)+(~b))*(~y)*(~z))
firstOrderRule = @acrule((~a)*(~y) + (~b)*(~y) => ((~a)+(~b))*(~y))
rules = Chain([firstOrderRule,secondOrderRule])
H1 = Fixpoint(rules)(H)
#H1 = Fixpoint(secondOrderRule)(H)
#H2 = Fixpoint(firstOrderRule)(H1)

(((-20.0 + 0.0im)g*ϵ_d*exp((0 + 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / (4(ω_d^2)) + ((-20.0 + 0.0im)g*ϵ_d*exp((0 - 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / (4(ω_d^2)) + ((-828.0 + 0.0im)g*ϵ_d*exp((0 - 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4 + ((-828.0 + 0.0im)g*ϵ_d*exp((0 + 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4 + ((-24.0 + 0.0im)g*ϵ_d*(τ^2)*exp((0 - 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4 + ((-24.0 + 0.0im)g*ϵ_d*(τ^2)*exp((0 + 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4*(σ_{1}12)+((-20.0 + 0.0im)g*ϵ_d*exp((0 + 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / (4(ω_d^2)) + ((-20.0 + 0.0im)g*ϵ_d*exp((0 - 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / (4(ω_d^2)) + ((-828.0 + 0.0im)g*ϵ_d*exp((0 - 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4 + ((-828.0 + 0.0im)g*ϵ_d*exp((0 + 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4 + ((-24.0 + 0.0im)g*ϵ_d*(τ^2)*exp((0 - 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4 + ((-24.0 + 0.0im)g*ϵ_d*(τ^2)*exp((0 + 1im)*t*ω_d)*exp(-0.5(τ^2)*(ω_d^2))) / 4*(σ_{1}21)+((-10.0 + 0.0im)ω_1*ϵ_d*exp((0 - 1im)*t*ω_d

In [37]:
k=2
γ, J_ops, Jd_ops = effective_dissipator(k, ωs, hs,t)
#print(5)

(Any[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0, 0, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0, 0  …  0, 0, 0, 0, 0.0 + 0.0im, 0, 0, 0, 0.0 + 0.0im, 0], Any[(ω_1+-ω_1*(σ_{1}22)+-0.5ω_1), (ω_2+-ω_2*(σ_{2}22)+-0.5ω_2), (g*(σ_{1}12*σ_{2}12)+g*(σ_{1}12*σ_{2}21)+g*(σ_{1}21*σ_{2}12)+g*(σ_{1}21*σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21)), (ω_1+-ω_1*(σ_{1}22)+-0.5ω_1), (ω_2+-ω_2*(σ_{2}22)+-0.5ω_2), (g*(σ_{1}12*σ_{2}12)+g*(σ_{1}12*σ_{2}21)+g*(σ_{1}21*σ_{2}12)+g*(σ_{1}21*σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21))  …  (ω_1+-ω_1*(σ_{1}22)+-0.5ω_1), (ω_2+-ω_2*(σ_{2}22)+-0.5ω_2), (g*(σ_{1}12*σ_{2}12)+g*(σ_{1}12*σ_{2}21)+g*(σ_{1}21*σ_{2}12)+g*(σ_{1}21*σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21)), (ω_1+-ω_1*(σ_{1}22)+-0.5ω_1), (ω_2+-ω_2*(σ_{2}22)+-0.5ω_2), (g*(σ_{1}12*σ_{2}12)+g*(σ_{1}12*σ_{2}21)+g*(σ_{1}21*σ_{2}12)+g*(σ_{1}21*σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21)), (ϵ_d*(σ_{2}12)+ϵ_d*(σ_{2}21))], Any[(ω_1+-ω_1*(σ_{1}22)

In [24]:
length(Jd_ops)

275

In [39]:
#Get meanfield equations using QuantumCumulants
eqs = meanfield([σz1, σz2, σx1, σx2, σx1*σz2] , H1; order=2,iv=t)

∂ₜ(1 - 2⟨σ_{1}22⟩) = ((0.0 - 4608.0im)*g*(ω_d^8)*⟨σ_{1}21*σ_{2}12⟩ + (0.0 - 4608.0im)*g*(ω_d^8)*⟨σ_{1}21*σ_{2}21⟩ + (0.0 + 4608.0im)*g*(ω_d^8)*⟨σ_{1}12*σ_{2}12⟩ + (0.0 + 4608.0im)*g*(ω_d^8)*⟨σ_{1}12*σ_{2}21⟩ + (0.0 - 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}21⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}12⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}12⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}21⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}12⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}12⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 3072.0im)*g*ϵ_d*(τ^2)*(ω_d^8)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 3072.0im)*g*ϵ_d*(τ^2)*

In [28]:
eqs = complete(eqs)

MethodError: MethodError: Cannot `convert` an object of type SymbolicUtils.Term{QuantumCumulants.AvgSym, Nothing} to an object of type SymbolicUtils.Add{CNumber, Int64, Dict{Any, Number}, Nothing}
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/essentials.jl:218
  SymbolicUtils.Add{X, T, D, M}(::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) where {X<:Number, T<:Number, D, M} at ~/.julia/packages/SymbolicUtils/qulQp/src/types.jl:617